# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka)** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Install Kafka SERVER

Go to `kafka` folder and run:

`docker compose up -d`

# Create SparkSession
## Install Kafka connector

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://0ad836d888db:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-09cee7f7-805f-4496-82bc-ad0932efc711;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

# Create a data stream from a Kafka topic

In [2]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()

# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")

# Send transformed data to the Sink
query_a = df_input.writeStream \
            .trigger(processingTime='1 second') \
            .outputMode("append") \
            .format("console") \
            .start()

25/10/10 01:02:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d4730423-5f0d-4312-a4cb-49574b1e385a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 01:02:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



25/10/10 01:02:27 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 3905 milliseconds
                                                                                

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
| hola|
+-----+



25/10/10 01:02:42 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 4471 milliseconds
                                                                                

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
| yeah|
+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+
|value|
+-----+
| yolo|
+-----+



25/10/10 01:03:00 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 1032 milliseconds


In [3]:
query_a.stop()
sc.stop()

25/10/10 01:03:52 WARN DAGScheduler: Failed to cancel job group 24c03544-73b8-4530-8046-fd0440459914. Cannot find active jobs for it.
25/10/10 01:03:52 WARN DAGScheduler: Failed to cancel job group 24c03544-73b8-4530-8046-fd0440459914. Cannot find active jobs for it.


## Add a Streaming listener

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pcamarillor.streaming_listener import MyListener

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://0ad836d888db:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Add custom listener
spark.streams.addListener(MyListener())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b3f4bbbe-d6b9-4521-860d-7767746f978c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

Query started: 305b910d-55bc-4652-a77f-c87421ce52bb
Query made progress: {
  "id" : "305b910d-55bc-4652-a77f-c87421ce52bb",
  "runId" : "39716853-24a2-469e-8855-ac57f4766232",
  "name" : null,
  "timestamp" : "2025-10-10T01:26:05.511Z",
  "batchId" : 0,
  "batchDuration" : 8652,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "addBatch" : 6083,
    "commitOffsets" : 321,
    "getBatch" : 34,
    "latestOffset" : 985,
    "queryPlanning" : 1129,
    "triggerExecution" : 8646,
    "walCommit" : 70
  },
  "stateOperators" : [ {
    "operatorName" : "stateStoreSave",
    "numRowsTotal" : 0,
    "numRowsUpdated" : 0,
    "allUpdatesTimeMs" : 94,
    "numRowsRemoved" : 0,
    "allRemovalsTimeMs" : 0,
    "commitTimeMs" : 385,
    "memoryUsedBytes" : 1200,
    "numRowsDroppedByWatermark" : 0,
    "numShufflePartitions" : 5,
    "numStateStoreInstances" : 5,
    "customMetrics" : {
      "loadedMapCacheHitCount" : 0,
      "loadedMap

In [4]:
from pyspark.sql.functions import explode, split
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()


# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")
df_input = df_input.select(explode(split(df_input.value, " ")).alias("word"))
df_input = df_input.groupBy("word").count()


# Send transformed data to the Sink
query_b = df_input.writeStream \
            .trigger(processingTime='3 second') \
            .outputMode("complete") \
            .format("console") \
            .start()


query_b.awaitTermination(25)

25/10/10 01:26:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-26b2b6ec-11c9-4942-8663-6ac32b977295. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 01:26:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



25/10/10 01:26:14 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000} milliseconds, but spent 8706 milliseconds
                                                                                

-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|hola|    1|
+----+-----+



25/10/10 01:26:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000} milliseconds, but spent 5150 milliseconds


False

-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|lesgo�|    1|
|  hola|    1|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|lesgo�|    1|
|yeahhh|    1|
|  hola|    1|
+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
| lesgo�|    1|
| yeahhh|    1|
|stardew|    1|
|   hola|    1|
+-------+-----+



25/10/10 02:11:32 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 1385998 ms exceeds timeout 120000 ms
25/10/10 02:11:34 ERROR TaskSchedulerImpl: Lost executor 0 on 172.20.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/10/10 02:11:34 ERROR Inbox: Ignoring error
java.lang.AssertionError: assertion failed: BlockManager re-registration shouldn't succeed when the executor is lost
	at scala.Predef$.assert(Predef.scala:279)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:741)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:141)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:104)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:216)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:101)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache

In [ ]:
query_b.stop()
sc.stop()